Installing Requirements

In [61]:
pip install snowflake-snowpark-python

In [62]:
pip install config

Import the required packages and modules

In [63]:
from snowflake.snowpark.session import Session

In [64]:
from config import *

Declare function to connect with snowpark

In [65]:
def snowpark_session():
  conn_params = {
      "account" : "pr91731-production_northeurope",
      "user" : input("Username : "),
      "password" : input("Password : "),
      "role" : "DATAENGINEER",
      "warehouse" : "INGESTION_WH"
  }

  session = Session.builder.configs(conn_params).create()

  return session

Initialize Snowpark Session

In [66]:
snowpark = snowpark_session()

column_wise_details = {}
missing_value = {}
unique = {}

Username : PRASANNARAVIKUMAR
Password : 241020@Nr


Run a sample script and create a dataframe from the result

In [67]:
data = snowpark.sql('SELECT * FROM LANDING_ZONE.SALESFORCE.ACCOUNT')

data.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [68]:
for col in data.columns:
  column_wise_details[col] = {"MISSING_VALUE_COUNT":data.filter(data[col].isNull()).count()}

# column_wise_details

In [69]:
ddl = snowpark.sql(f"DESCRIBE TABLE LANDING_ZONE.SALESFORCE.ACCOUNT").collect()

for row in ddl:
  column_wise_details[row["name"]]["DATA_TYPE"] = row["type"]

# column_wise_details

In [70]:
for column in data.columns:
  unique[column] = int(data.select(column).distinct().to_pandas().shape[0])-1

unique

{'ID': 62683,
 'IS_DELETED': 1,
 'MASTER_RECORD_ID': 80,
 'NAME': 61726,
 'TYPE': 71,
 'PARENT_ID': 2820,
 'BILLING_STREET': 6997,
 'BILLING_CITY': 1425,
 'BILLING_STATE': 97,
 'BILLING_POSTAL_CODE': 3351,
 'BILLING_COUNTRY': 98,
 'BILLING_STATE_CODE': 88,
 'BILLING_COUNTRY_CODE': 98,
 'BILLING_LATITUDE': 1,
 'BILLING_LONGITUDE': 1,
 'BILLING_GEOCODE_ACCURACY': 0,
 'SHIPPING_STREET': 6578,
 'SHIPPING_CITY': 1370,
 'SHIPPING_STATE': 89,
 'SHIPPING_POSTAL_CODE': 3185,
 'SHIPPING_COUNTRY': 95,
 'SHIPPING_STATE_CODE': 82,
 'SHIPPING_COUNTRY_CODE': 95,
 'SHIPPING_LATITUDE': 0,
 'SHIPPING_LONGITUDE': 0,
 'SHIPPING_GEOCODE_ACCURACY': 0,
 'PHONE': 12480,
 'FAX': 1099,
 'ACCOUNT_NUMBER': 0,
 'WEBSITE': 9693,
 'PHOTO_URL': 62683,
 'SIC': 0,
 'INDUSTRY': 105,
 'ANNUAL_REVENUE': 1,
 'NUMBER_OF_EMPLOYEES': 0,
 'OWNERSHIP': 0,
 'TICKER_SYMBOL': 0,
 'DESCRIPTION': 2274,
 'RATING': 1,
 'SITE': 0,
 'CURRENCY_ISO_CODE': 15,
 'OWNER_ID': 596,
 'CREATED_DATE': 4840,
 'CREATED_BY_ID': 311,
 'LAST_MODIFIED_

In [71]:
pip install matplotlib

In [72]:
for column in column_wise_details.keys():
  print(data[column]["type"])
  if "NUMBER" in data[column][type] or "FLOAT" in data[columns]["type"]:
    try:
       = data.approx_quantile(column,[0.25,0.75])

      column_wise_details[column]["Q1"] = Q1 = quartile[0]
      column_wise_details[column]["Q3"] = Q3 = quartile[1]

      IQR = Q3-Q1
quartile
      column_wise_details[column]["IQR"] = IQR

      column_wise_details[column]["LOWER_LIMIT"] = lower_limit = Q1 - 1.5 * IQR
      column_wise_details[column]["UPPER_LIMIT"] = upper_limit = Q3 + 1.5 * IQR

      column_wise_details[column]["OUTLIERS"] = outliers = data.filter((data[column] < lower_limit) | (data[column] > upper_limit)).count()

      column_wise_details[column]["UNIQUE_COUNT"] = unique_count = unique[column]


    except Exception as e:
      print(e)

"""
for column_name in column_wise_details.keys():
  print(column_name,end = " -> ")
  column_details = column_wise_details[column_name]
  for details in column_details.keys():
    print(details,":",column_details[details],end=",")
  print()
"""
# print(column_wise_details)

Column[SUBFIELDSTRING]


TypeError: ignored

In [ ]:
import openpyxl
from google.colab import files

In [ ]:
column_wise_details

In [ ]:
largest_dict = {}
for col in column_wise_details:
  for var in column_wise_details[col].keys():
    if var not in excel_columns:
      excel_columns.append(var)

excel_columns

In [ ]:
workbook = openpyxl.Workbook()

worksheet = workbook.active

row = 2
for excel_column_name in excel_columns:
  worksheet.cell(row = 1, column = excel_columns.index(excel_column_name)+1, value = excel_column_name)

for column_name in column_wise_details.keys():
  # print(column_name,end=" > ")
  for excel_column_name in excel_columns:
    # print(excel_column_name,end=":")
    if(excel_column_name == "Column_Name"):
      worksheet.cell(row = row, column = excel_columns.index(excel_column_name)+1, value = column_name)
      # print(column_name,end=", ")
    else:
      if(excel_column_name in column_wise_details[column_name].keys()):
        worksheet.cell(row = row, column = excel_columns.index(excel_column_name)+1, value = column_wise_details[column_name][excel_column_name])
        # print(column_wise_details[column_name][excel_column_name],end=", ")
      else:
        continue
  row+=1
  # print()


workbook.save("DataQuality.xlsx")

files.download("DataQuality.xlsx")

In [ ]:
from scipy.stats import norm
import numpy as np
import pandas as pd

pandas_df = data.toPandas()

pandas_df

pd_data = pd.to_numeric(pandas_df["ACCOUNT_VALUE_C"],errors = 'coerce').dropna()

mean = np.mean(pd_data)
std = np.std(pd_data)

x = np.linspace(min(pd_data),max(pd_data),100)
y= norm.pdf(x,mean,std)

plt.style.use('seaborn')
plt.figure(figsize = (6,6))
plt.plot(x,y,color = 'black',linestyle = 'dashed')
plt.scatter(x,y,marker = 'o',s=25,color = 'red')
plt.show()